In [16]:
import os 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from glob import glob 
import json 

exp_list= glob('./*')

metrics = ['Image AUC', 'Pixel AUC', 'Image AP', 'Pixel AP']
df = pd.DataFrame()
for exp_path in exp_list:
    for log_path in glob(os.path.join(exp_path, 'final_results.csv')):
        temp_df = pd.read_csv(log_path)
        # metrics 컬럼을 float로 변환
        for m in metrics:
            if m in temp_df.columns:
                temp_df[m] = pd.to_numeric(temp_df[m], errors='coerce')
        temp_df['exp_name'] = os.path.basename(exp_path)
        df = pd.concat([df, temp_df], ignore_index=True)


In [17]:
# 'Class Name'이 'Overall'이 아닌 행들만 남기고,
# exp_name에서 dataset을 추출하여 dataset별로 실험들을 비교합니다.
# exp_name은 텍스트 모두 표시되도록(줄바꿈 없이, 컬럼 너비 제한 없이) display 옵션을 조정함
import pandas as pd

metrics = ['Image AUC',  'Pixel AP']

df_filtered = df.copy()
for m in metrics:
    df_filtered[m] = pd.to_numeric(df_filtered[m], errors='coerce')

# exp_name에서 dataset 추출 (첫 번째 하이픈 또는 언더스코어로 분리)
def extract_dataset(exp_name):
    """exp_name에서 dataset을 추출합니다."""
    if pd.isna(exp_name):
        return 'Unknown'
    exp_str = str(exp_name)
    # 하이픈으로 분리 시도
    if '-' in exp_str:
        return exp_str.split('-')[0]
    # 언더스코어로 분리 시도
    elif '_' in exp_str:
        return exp_str.split('_')[0]
    # 분리자가 없으면 전체를 dataset으로 간주
    else:
        return exp_str

# 현재 task id 추출 함수 - 폴더 내 continual_results_after_task_@.csv 파일을 보고 파악 
def extract_current_task(exp_folder):
    """
    exp_folder 내에서 현재 몇 번째 Task를 학습 중인지 파악.
    - 'continual_results_after_task_@.csv' 파일명을 탐색해서 가장 큰 @ 추출
    - 해당 파일이 없으면 None 반환
    """
    import os
    from glob import glob
    import re

    pattern = os.path.join(exp_folder, "continual_results_after_task_*.csv")
    files = glob(pattern)
    task_nums = []
    for f in files:
        m = re.search(r"continual_results_after_task_(\d+)\.csv", os.path.basename(f))
        if m:
            task_nums.append(int(m.group(1)))
    if task_nums:
        return max(task_nums)
    else:
        return None

df_filtered['dataset'] = df_filtered['exp_name'].apply(extract_dataset)

# dataset별 exp_name별 전체 평균 (각 실험별 전체 평균)
df_exp_mean = df_filtered.groupby(['dataset', 'exp_name'])[metrics].mean().reset_index()

# Average 컬럼 추가 (Image AUC, Pixel AP의 평균)
df_exp_mean['Average'] = df_exp_mean[metrics].mean(axis=1)

# Average 기준 내림차순 정렬 후 상위 10개 실험만 남기기
df_top10 = df_exp_mean.sort_values('Average', ascending=False).groupby('dataset').head(15).reset_index(drop=True)

# 현재 진행 중인(폴더만 있고 결과가 아직 없는) 실험 찾아서 표시
import os
from glob import glob

# 현재 디렉토리 내 모든 실험 폴더 탐색 (디렉토리만)
exp_dirs = [d for d in glob('*') if os.path.isdir(d)]
running_exps = []
finished_exps = []

for d in exp_dirs:
    csv_path = os.path.join(d, 'final_results.csv')
    if not os.path.isfile(csv_path):
        running_exps.append(d)
    else:
        finished_exps.append(d)

# --- exp_name 텍스트 전부 표시 설정 ---
pd.set_option('display.max_colwidth', None)

# dataset별 클래스별 결과 비교 (피벗 테이블: index=Class Name, columns=exp_name, grouped by dataset)
datasets = df_filtered['dataset'].unique()
for dataset in sorted(datasets):
    print(f"\n{'='*80}")
    print(f"Dataset: {dataset}")
    print(f"{'='*80}")

    # 해당 dataset의 데이터만 필터링
    df_dataset = df_filtered[df_filtered['dataset'] == dataset]

    # 해당 dataset의 exp_name별 클래스별 결과
    df_class_compare = df_dataset.pivot_table(index='Class Name', columns='exp_name', values=metrics)

    print(f"\n[{dataset} - 실험별 전체 평균 (Average 상위 10개만)]")
    display(df_top10[df_top10['dataset'] == dataset])

    print(f"\n[{dataset} - 클래스별 비교]")
    display(df_class_compare)
    
    # 진행 중인 실험(폴더만 있음) 중, 해당 dataset에 해당하는 것도 표시
    # exp 폴더 이름이 'dataset-' 또는 'dataset_'로 시작하는 것에 한정
    running_this_dataset = []
    for exp_folder in running_exps:
        if exp_folder.startswith(dataset + "-") or exp_folder.startswith(dataset + "_") or exp_folder == dataset:
            running_this_dataset.append(exp_folder)
    if running_this_dataset:
        print(f"\n[{dataset} - 현재 진행 중(폴더만 있고 결과없는) 실험 디렉토리]:")
        for exp in running_this_dataset:
            # 현재 task 정보 추출
            task_num = extract_current_task(exp)
            if task_num is not None:
                print(f"  진행중: {exp} (현재 Task {task_num} 학습 중)")
            else:
                print(f"  진행중: {exp} (현재 Task 정보 없음)")




Dataset: MPDD

[MPDD - 실험별 전체 평균 (Average 상위 10개만)]


,dataset,exp_name,Image AUC,Pixel AP,Average
15,MPDD,MPDD-WRN50-60ep-lr2e4-dia4,0.901883,0.288983,0.595433



[MPDD - 클래스별 비교]


,Image AUC,Pixel AP
exp_name,MPDD-WRN50-60ep-lr2e4-dia4,MPDD-WRN50-60ep-lr2e4-dia4
Class Name,,
bracket_black,0.7427,0.0108
bracket_brown,1.0000,0.0133
bracket_white,0.7844,0.0020
connector,0.9929,0.2948
metal_plate,1.0000,0.9003
tubes,0.8913,0.5127



Dataset: MVTec

[MVTec - 실험별 전체 평균 (Average 상위 10개만)]


...

Image AUC                                               \
exp_name   MVTec-ViT-60ep MVTec-WRN50-60ep-lr2e4-dia4 MVTec-WRN50-80ep   
Class Name                                                               
bottle             1.0000                      1.0000           1.0000   
cable              0.9350                      0.9884           0.9933   
capsule            0.7495                      0.9609           0.9569   
carpet             0.9759                      0.9835           0.9835   
grid               0.9315                      0.9983           0.9908   
hazelnut           0.9614                      0.9993           1.0000   
leather            1.0000                      1.0000           1.0000   
metal_nut          0.9804                      1.0000           1.0000   
pill               0.8718                      0.9888           0.9839   
screw              0.4122                      0.9094           0.9035   
tile               1.0000                      1.0000           1.0000   
toothbrush         0.8972                      0.8861           0.9000   
transistor         0.8154                      0.9950           1.0000   
wood               0.9675                      0.9842           0.9904   
zipper             0.9315                      0.9961           0.9916   

                                                                        \
exp_name   MVTec-WRN50-Ablation-wo_Adapter MVTec-WRN50-Ablation-wo_DIA   
Class Name                                                               
bottle                              1.0000                      1.0000   
cable                               0.9846                      0.9783   
capsule                             0.8672                      0.8077   
carpet                              0.9876                      0.9912   
grid                                0.9825                      0.9758   
hazelnut                            1.0000                      0.9996   
leather                             1.0000                      1.0000   
metal_nut                           0.9995                      0.9985   
pill                                0.9716                      0.9626   
screw                               0.7530                      0.6661   
tile                                1.0000                      1.0000   
toothbrush                          0.8806                      0.8833   
transistor                          1.0000                      0.9971   
wood                                0.9895                      0.9877   
zipper                              0.9898                      0.9709   

                                                                          \
exp_name   MVTec-WRN50-Ablation-wo_LoRA MVTec-WRN50-Ablation-wo_PosEmbed   
Class Name                                                                 
bottle                           1.0000                           1.0000   
cable                            0.9918                           0.9888   
capsule                          0.9513                           0.9509   
carpet                           0.9868                           0.9900   
grid                             0.9875                           0.9875   
hazelnut                         1.0000                           1.0000   
leather                          1.0000                           1.0000   
metal_nut                        1.0000                           1.0000   
pill                             0.9825                           0.9831   
screw                            0.9006                           0.8817   
tile                             1.0000                           1.0000   
toothbrush                       0.9111                           0.9000   
transistor                       1.0000                           0.9962   
wood                             0.9912                           0.9860   
zipper                           0.9932                   


Dataset: VISA

[VISA - 실험별 전체 평균 (Average 상위 10개만)]


,dataset,exp_name,Image AUC,Pixel AP,Average
16,VISA,VISA-WRN50-LoRA128-DIA6-Combined,0.856558,0.276133,0.566346
17,VISA,VISA-WRN50-60ep-lr2e4-dia4,0.837833,0.287775,0.562804
18,VISA,VISA-WRN50-DIA6-80ep,0.837575,0.275025,0.556300
19,VISA,VISA-WRN50-80ep-lr3e4,0.827192,0.269783,0.548488
20,VISA,VISA-Ablation-LoRA128-DIA6-TailW0.7-TopK3,0.818217,0.265167,0.541692
21,VISA,VISA-ViT-60ep,0.880125,0.198217,0.539171
22,VISA,VISA-WRN50-LoRA128-80ep,0.820200,0.257133,0.538667
23,VISA,VISA-Default-TailW0.7-TopK3-TailTopK2-ScaleK5-lr3e-4,0.810675,0.260817,0.535746
24,VISA,VISA-Ablation-TailW0.8-TopK3-TailTopK2-ScaleK5,0.811367,0.259033,0.535200



[VISA - 클래스별 비교]


Image AUC  \
exp_name   VISA-Ablation-LoRA128-DIA6-TailW0.7-TopK3   
Class Name                                             
candle                                        0.8291   
capsules                                      0.6948   
cashew                                        0.8266   
chewinggum                                    0.8456   
fryum                                         0.9534   
macaroni1                                     0.7429   
macaroni2                                     0.6833   
pcb1                                          0.8328   
pcb2                                          0.7563   
pcb3                                          0.7909   
pcb4                                          0.9427   
pipe_fryum                                    0.9202   

                                                           \
exp_name   VISA-Ablation-TailW0.8-TopK3-TailTopK2-ScaleK5   
Class Name                                                  
candle                                             0.8169   
capsules                                           0.6888   
cashew                                             0.8164   
chewinggum                                         0.8253   
fryum                                              0.9552   
macaroni1                                          0.7359   
macaroni2                                          0.6859   
pcb1                                               0.8313   
pcb2                                               0.7529   
pcb3                                               0.7729   
pcb4                                               0.9443   
pipe_fryum                                         0.9106   

                                                                               \
exp_name   VISA-Default-TailW0.7-TopK3-TailTopK2-ScaleK5-lr3e-4 VISA-ViT-60ep   
Class Name                                                                      
candle                                                   0.8172        0.8932   
capsules                                                 0.6862        0.8692   
cashew                                                   0.8172        0.9726   
chewinggum                                               0.8246        0.9154   
fryum                                                    0.9564        0.9262   
macaroni1                                                0.7356        0.8419   
macaroni2                                                0.6886        0.6490   
pcb1                                                     0.8267        0.9234   
pcb2                                                     0.7544        0.8767   
pcb3                                                     0.7668        0.8121   
pcb4                                                     0.9446        0.9492   
pipe_fryum                                               0.9098        0.9326   

                                                             \
exp_name   VISA-WRN50-60ep-lr2e4-dia4 VISA-WRN50-80ep-lr3e4   
Class Name                                                    
candle                         0.8843                0.8200   
capsules                       0.6743                0.6967   
cashew                         0.8794                0.8474   
chewinggum                     0.9630                0.9542   
fryum                          0.9360                0.9526   
macaroni1                      0.7609                0.7375   
macaroni2                      0.6597                0.6748   
pcb1                           0.8428                0.8393   
pcb2                           0.7309                0.7369   
pcb3                           0.7845                0.7676   
pcb4                           0.9710                0.9495   
pipe_fryum                     0.9672                0.9498   

                                                         \
exp_name   VISA-WRN50-DIA6-80ep VISA-WRN50-LoRA128-80ep   
Class 